In [10]:
# Initialisation

import sqlite3

#Search criteria
NMI_CODE = '4103693793'
DATE_WANTED = '2016-03'

db_file = "test_data.db"
tb_metering = 'raw_metering'
tb_holidays = 'holidays'
tb_nmi_info = 'nmi_info'
tb_retailer_timeband = 'retailer_timeband'

In [3]:




#used PARAMS
NMI_State = ''
NMI_Tariff = ''
NMI_Retailer = ''
NMI_NetPro = ''

conn = sqlite3.connect(db_file)

c = conn.cursor()

# find State for nmi
c.execute(fetch_NMI_Info ('State', tb_nmi_info, NMI_CODE))
NMI_State = c.fetchone()[0]

# find tariff for nmi
c.execute(fetch_NMI_Info ('Tariff', tb_nmi_info, NMI_CODE))
NMI_Tariff = c.fetchone()[0]

# find Retailer for nmi
c.execute(fetch_NMI_Info ('Retailer', tb_nmi_info, NMI_CODE))
NMI_Retailer = c.fetchone()[0]

# find Network Provider for nmi
c.execute(fetch_NMI_Info ('Network_Provider', tb_nmi_info, NMI_CODE))
NMI_NetPro = c.fetchone()[0]


t = (NMI_Retailer,NMI_State)

query = """
SELECT *
FROM retailer_timeband
WHERE Retailer = ?
AND State = ?
"""
c.execute(query, t)

NMI_State = [x for x in c.fetchall()]

# end database
conn.close()


In [9]:
conn = sqlite3.connect(db_file)

c = conn.cursor()

# find tariff for nmi
holiday_query = "AND date(Reading_Date) != '2016-04-25'"

table = 'raw_metering'
query = '''
SELECT rowid, sum(Value)
FROM raw_metering
WHERE NMI_Suffix = 'E1'
AND strftime('%w', Reading_Date) BETWEEN '1' AND '5'
AND ((Time(Reading_Date) >= time('07:00') AND Time(Reading_Date) <time('09:00'))
OR (Time(Reading_Date) >= time('17:00')
AND Time(Reading_Date) <time('20:00')))
AND strftime('%m', Reading_Date) = '04'

''' + holiday_query

c.execute(query)
print c.fetchall()
print 'Done'





conn.close()


[(22704, 2649.5039999999985)]
Done


In [ ]:
conn = sqlite3.connect(db_file)

c = conn.cursor()

# find tariff for nmi
table = 'raw_metering'


query = """
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN substr(Reading_Date, 13,1) = ':'
THEN substr(Reading_Date, 1, 11) || '0' || substr(Reading_Date, 12, 4)
ELSE Reading_Date
END
"""


c.execute(query)
print 'Done'


conn.close()

# Functions

In [43]:
# create new table
def create_table():
    pass

# check for all tables in db
def find_all_tables():
    pass

# update table
def update_table():
    pass

# import csv file
def import_csv():
    pass

# export csv file
def export_csv():
    pass

def correct_time_stype(time_string):
    key = ":"
    if time_string.find(key) == 1:
        return '0' + time_string
    else:
        return time_string
def fetch_NMI_Info(field1, table1, NMI1):
    query = '''
        SELECT %s
        FROM %s
        WHERE NMI = %s
        '''
    return query % (field1, table1, NMI1)

def general_query(q_info):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute(q_info)
    data = c.fetchall()
    c.close
    return data

# return holiday list
def holiday_query(q_state, q_target, q_mth, q_yr):
    """
    this will return a list of holidays of the given state, network/retailer, months
    """
    query = """
        SELECT Date
        FROM holidays
        WHERE State = '%s' 
        AND Target = '%s'

    """
    #AND strftime('%m', Date) = %s 
    #AND strftime('%Y', Date) = %s
    sub_query = query % (q_state, q_target)
    #print sub_query
    return general_query(sub_query)

In [46]:
print 'hello',holiday_query ('NSW', 'Network', '03', '2016')[0]

hello (u'2016-01-01',)


In [11]:
test_val = "0:12"
print correct_time_stype(test_val)


00:12


In [ ]:
# update datetime column
UPDATE raw_metering SET Reading_Date = replace(Reading_Date, '/', '-')

SELECT (substr(Reading_Date, 7, 4) || '-' || substr(Reading_Date, 4, 2) || '-' || substr(Reading_Date, 1, 2) as dd)
FROM raw_metering 

# update date formate
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN Instr(Reading_Date,'/') =2
THEN substr(Reading_Date, 6, 4) || '-' || substr(Reading_Date, 3, 2) || '-0' || substr(Reading_Date, 1, 1) || substr(Reading_Date, Instr(Reading_Date,' '), 6)
WHEN Instr(Reading_Date,'/') =3 
THEN (substr(Reading_Date, 7, 4) || '-' || substr(Reading_Date, 4, 2) || '-' || substr(Reading_Date, 1, 2)) || substr(Reading_Date, Instr(Reading_Date,' '), 6)
ELSE Reading_Date
END

# update time formate
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN Instr(Reading_Date, ':') = 13  
THEN substr(Reading_Date, 1, 11) || '0' || substr(Reading_Date, 12, 4)
ELSE Reading_Date
END

#
SELECT SUM(Value)
FROM raw_metering
WHERE NMI_Suffix = 'E1' and Quality_Method = 'A' and strftime('%w', Reading_Date) between '0' and '6'

#peak1_start
SELECT Peak1_Start FROM
retailer_timeband,
(SELECT (nmi_info.State) as state1 FROM nmi_info WHERE NMI = '4103693793') as sub_state,
(SELECT (nmi_info.Retailer) as retail FROM nmi_info WHERE NMI = '4103693793') as sub_retail
WHERE retailer_timeband.Retailer = retail AND retailer_timeband.State = state1